In [2]:
import docx
import base64
import os
import openai

In [3]:
INPUT_CTX_PATH = "input/ctx"
FEATURE_PATH = "input/feature"
OUTPUT_PATH = "output"

In [6]:
def handledocx(filepath):
    doc = docx.Document(filepath)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    resp = {
      "type": "text",
      "text": '\n'.join(fullText)
    }
    return resp


def handlepng(filepath):

    with open(filepath, "rb") as image_file:
         base64_image = base64.b64encode(image_file.read()).decode('utf-8')
    
    resp =   {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
    }
    return resp
    

In [7]:
import os

# print(os.listdir(INPUT_PATH))
supported_files = {
    "png": handlepng,
    # "pdf": "handlepdf",
    "docx": handledocx,
    # "txt": "handle txt",
}


def getcontent(path):
    content = []
    input_files = [f for f in os.listdir(path) if os.path.isfile(f'{path}/{f}')]
    
    for i in input_files:
    
        ext = i.split(".")[-1]
        try:
            inp = supported_files[ext](f'{path}/{i}')
            content.append(inp)
        except Exception as e:
            print(f"unsupported file type, {i}, {e}")
    return content

def promptobj(prompt):
    return {
      "type": "text",
      "text": prompt
    }

pre_prompt = promptobj("You are a product tester who will write test cases for the following project.")

ctx_content = getcontent(INPUT_CTX_PATH)

mid_prompt = promptobj("For the above product we are building a new feature with the below description.")

feature_content = getcontent(FEATURE_PATH)

end_prompt = promptobj("Generate 50 test cases for this new feature of the format, (test_case_name, test_case_description) for this products. Do not say anything else. Only provide csv compatible text output. and do not generate less than 50 test cases")


unsupported file type, RoadTrip.pdf, 'pdf'


In [8]:
import base64
import requests

# OpenAI API Key
with open("secrets.txt","r") as f:
    api_key = f.read()


# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "path_to_your_image.jpg"



headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}



payload = {
  "model": "gpt-4-vision-preview",
  "messages": [
    {
      "role": "user",
      "content": [pre_prompt,*ctx_content, mid_prompt, *feature_content, end_prompt]
    }
  ],
  "max_tokens": 3000
}

# print(payload)

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())
    

{'id': 'chatcmpl-9OzXoBZEPyUJX39CM08C3J0lXyRSy', 'object': 'chat.completion', 'created': 1715743476, 'model': 'gpt-4-1106-vision-preview', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "test_case_name,test_case_description\nVR_Scenario_Selection_001,Verify that the VR environment correctly loads a pre-designed scenario.\nVR_Scenario_Selection_002,Verify that the user can scroll through and select from a list of scenarios.\nVR_Scenario_Selection_003,Validate that the VR environment displays an error when an invalid scenario is selected.\nVR_Scenario_Selection_004,Verify custom scenario creation with valid recorded dialogue.\nVR_Scenario_Selection_005,Test loading times for scenario selection on various devices.\nVR_Environment_Visuals_001,Ensure that visuals match the selected scenario (coffee shop, airport, etc.).\nVR_Environment_Visuals_002,Verify visual quality and resolution in the VR environment.\nVR_Environment_Visuals_003,Check for visual glitches or artifa

In [9]:
import datetime

now = datetime.datetime.now()

with open(f'{OUTPUT_PATH}/output1.csv', "w+") as f:
    f.write(response.json()['choices'][0]['message']['content'])